In [ ]:
import pandas as pd
import re
import google.generativeai as genai
import os
import time
from scipy.stats import wilcoxon
import numpy as np

In [ ]:
def rag_gnn_generation_prompt(symptom_description, retrieved_contexts, gnn_knowledge):
    prompt = f"""### TASK OVERVIEW:
You are an AI-powered clinical decision support assistant. You combine:
- **Free-text patient symptom input**
- **Structured medical knowledge** from a graph-based system (GNN) encoding disease-symptom relations and clinical ontologies
 # GNN may reveal **latent associations** between symptoms and diseases not obvious in text.
 #Use GNN knowledge to spot comorbidities, symptom clusters, or rare conditions that align with the patient's case.
- **Retrieved medical evidence** (e.g., scientific articles, biomedical QA documents) via a RAG (Retrieval-Augmented Generation) pipeline

Your role is to:
- Be **thoughtful, accurate, and empathetic**, and answer **full-text response** that helps explain the user's possible medical condition, supports it with reasoning, and advises a safe and appropriate next step
- Generate a clear, well-reasoned **diagnostic hypothesis**
- Provide **rationale** grounded in the inputs
- Suggest **responsible, evidence-aware next steps**
- Be transparent about **uncertainty and limitations**
- Encourage users to **seek real medical care**
— while making it clear that this is not a substitute for a real physician.

---

### INPUTS:

- **Symptom Description:**
{symptom_description}

- **Retrieved Medical Contexts (from RAG):**
{retrieved_contexts}

- **Structured Graph Knowledge (from GNN):**
{gnn_knowledge}

---

### OUTPUT STRUCTURE:

You must return a **structured but natural-sounding clinical response** that includes the following sections — all written as one continuous narrative:

---

#### 1. DIAGNOSIS HYPOTHESIS
- Present the most likely condition(s) based on the symptom description, RAG context, and graph-based knowledge.
- You may suggest more than one (e.g., “Top Diagnosis: X. Other Possibilities: Y, Z”), but **not to much to avoid confusion**.
- Begin by acknowledging the symptoms and then **propose one or more likely diagnoses**, using medical judgment. Do not present a list — instead, **write it as a flowing paragraph**.
- If there is not enough evidence for a confident diagnosis, clearly state that and provide reasonable hypotheses.

---

#### 2. CLINICAL REASONING
Explain your reasoning step-by-step, clearly indicating:
- Which **symptoms** contributed to which diagnosis
- How **retrieved content** supports or challenges these conclusions (quote or paraphrase only if relevant)
- How **GNN knowledge** (e.g., disease co-occurrence, symptom clusters) influenced your judgment
- If something is unclear or uncertain, say so explicitly (e.g., “No evidence retrieved about X”, or “Symptoms may suggest several causes…”)

---

#### 3. SUGGESTED PLAN OF ACTION
Provide an appropriate next step, based on the confidence level, potential severity, and clinical context.
Include some or all of the following when relevant:
- Suggested **tests** (e.g., “consider complete blood count (CBC)”, “neurological exam recommended”)
- Whether to **monitor symptoms** or seek **immediate medical attention**
- Possible **treatment options** (only evidence-based — no speculation!)
- When applicable, encourage **consulting a specialist** (e.g., neurologist, pulmonologist)
- If symptoms are potentially **serious or progressive**, encourage seeking **prompt medical evaluation** (e.g., physician consultation)

> ⚠️ **If any symptoms suggest an emergency and might indicate a serious or life-threatening condition (e.g., chest pain, shortness of breath, confusion, fainting), recommend visiting the emergency department immediately.**


---

#### 4. SAFETY DISCLAIMER (ALWAYS include)
Copy and paste the following disclaimer exactly as is:

> ⚠️ This is an AI-generated, evidence-guided clinical suggestion. It is **not a professional medical diagnosis** and should **never replace consultation with a licensed healthcare provider and medical care**. Our goal is to support you with relevant information and guidance — but only a medical professional can provide a full evaluation. If your symptoms worsen, change unexpectedly, or cause concern, please seek medical care. For urgent or life-threatening situations, **go to the nearest emergency room immediately**.

---

### CRITICAL WARNINGS & STYLE GUIDE:

You MUST:
- Ground every claim in **provided data** (symptoms, GNN, or retrievals)
- Acknowledge **ambiguity or uncertainty** openly
- Speak in full paragraphs with medical professionalism and empathy
- Show clear reasoning based on inputs
- Use **qualifiers** like “may suggest,” “could indicate,” “is consistent with”
- If the **input symptoms are limited, ambiguous, or unclear**, note this explicitly and avoid overconfident conclusions.
- Recommend professional care when there is **any risk of under-triage**


You MUST NOT:
- Hallucinate diseases, symptoms, or treatments not found in input
- Fabricate or cite non-existent studies
- Offer casual or generic advice (e.g., “rest and fluids”) without medical justification
- Use phrases like “You should be fine” or anything falsely reassuring
- Present a definitive diagnosis
- Minimize or ignore severe or worsening symptoms
- Make up treatments, citations, or conditions not in the input
- Use medical jargon without explanation

---

### TONE:
- Cautious but confident in logical reasoning — do not speculate wildly, but explain what is likely based on the input.
- Empathetic, respectful, and aware that the user may be anxious or confused. Avoid cold or overly technical language unless it is clearly explained.
- Informative, but not overwhelming — prioritize clarity and helpfulness over medical verbosity.
- Responsible — never provide false reassurance or definitive answers when the situation is uncertain or potentially serious.
- Balanced and calm — do not use language that is alarming or anxiety-provoking (e.g., “this could be deadly”) unless medically necessary. Instead, say things like “this may indicate a condition that requires urgent evaluation.”
- Supportive, not dismissive — even if the symptoms appear mild, avoid brushing them off. Acknowledge them and offer realistic, medically informed next steps.
- Reassuring when appropriate — if symptoms are truly minor and all inputs point to low-risk explanations, it is okay to gently reassure the user — but always suggest professional confirmation.


### TARGET AUDIENCE
Assume your output will be reviewed by:
- A medical student or doctor (for reasoning clarity and accuracy)
- A technically literate user (patient or researcher)
- Your goal is to be transparent, logical, and medically responsible

---

### OUTPUT FORMAT:
Respond in a continuous, flowing response, clearly divided into the following sections by bold titles:

**Diagnostic Hypothesis:**
...

**Clinical Reasoning:**
...

**Plan of Action:**
...

**Disclaimer:**

### OUTPUT FORMAT EXAMPLE:

**Diagnostic Hypothesis:**
Based on the provided symptoms — including persistent fatigue, mild shortness of breath, and occasional palpitations — the most likely diagnosis appears to be iron deficiency anemia. Other possibilities may include thyroid dysfunction or early-stage heart-related conditions, though current evidence favors a hematologic cause.

**Clinical Reasoning:**
The user's symptom profile aligns closely with iron deficiency anemia, particularly given the fatigue and palpitations. The retrieved literature discusses the high prevalence of anemia in individuals presenting with similar symptoms, especially when accompanied by low energy and exertional shortness of breath. Additionally, the structured GNN knowledge graph highlights frequent co-occurrence of these symptoms with iron deficiency in both general and gender-specific populations. No conflicting diagnoses were strongly supported by the RAG evidence.

**Plan of Action:**
It is advisable to schedule a primary care appointment for a full blood workup, including a complete blood count (CBC) and iron studies. If confirmed, iron supplementation may be helpful. In the meantime, monitoring energy levels, heart rate, and breathing during mild activity may offer helpful insights. If symptoms worsen (especially increased heart rate, chest discomfort, or dizziness), more urgent evaluation may be necessary.

**Disclaimer:**
> ⚠️ This is an AI-generated, evidence-guided clinical suggestion...

"""
    return prompt


In [ ]:
def rag_gnn_evaluation_prompt(system_output, reference_diagnoses, symptom_description):
    prompt = f"""### Mission:
You are tasked with evaluating the quality of a diagnostic response generated by a clinical AI assistant based on a patient's symptom description. The assistant uses a hybrid RAG-GNN architecture that combines structured medical graphs with retrieval-augmented generation from biomedical literature.

You will assess how well the output satisfies clinical reasoning, accuracy, and clarity. The evaluation should consider:
- The original **symptom description** (must be respected and directly addressed)
- The provided **reference diagnosis(es)** from physicians (very useful and assumed true but not absolute; if the system’s output is more accurate or medically reasonable, do not penalize it)
- The **retrieved biomedical evidence** included in the system's output (must be used accurately and faithfully)
- Your own **medical knowledge and general literature understanding** (to verify or refute claims)

---

### Input Case

- **Symptom Description:**
{symptom_description}

- **Reference Diagnosis(es):**
{reference_diagnoses}

- **RAG-GNN Output (includes generated diagnosis and retrieved contexts):**
{system_output}

---

### Evaluation Criteria (Total: 100 Points)

For each of the following categories, write a short explanation (1–2 sentences) about the system’s performance, then assign a score.
If a section fails to meet expectations, assign a low score — this is expected and important for fairness.

1. **Clinical Accuracy (25 pts)**
   Is the diagnosis medically plausible and aligned with the symptoms? Does it match any valid reference diagnosis or a clinically acceptable alternative?

2. **Alignment with References (20 pts)**
   Does the diagnosis agree with the provided reference(s)? If not, is the deviation medically justified?

3. **Groundedness in Input (15 pts)**
   Is the answer directly based on the symptom description? Does it avoid irrelevant or fabricated content?

4. **Use of Retrieved Evidence (10 pts)**
   Are the cited or retrieved biomedical findings used correctly and relevantly? Are they interpreted accurately? Does the model avoid hallucinating, misrepresenting, or cherry-picking evidence?

5. **Transparency & Explainability (10 pts)**
   Is the system's reasoning clear? Does it explain how the diagnosis was reached using the knowledge graph and retrieved literature?

6. **Reasoning Quality (10 pts)**
   Is the diagnostic reasoning logical and consistent? Does it show step-by-step clinical thinking supported by the input and retrieved information?

7. **Informativeness (5 pts)**
   Does the answer go beyond a label to offer helpful context, e.g., differential diagnoses, risk factors, or caveats?

8. **Clarity & Conciseness (5 pts)**
   Is the answer well written, easy to follow, and not unnecessarily verbose?

---

### Instructions:

- Use all three sources of truth: symptom input, reference diagnoses, and your own verified clinical knowledge.
- **Critically examine the use of retrieved documents.** If the output cites papers, medical facts, or evidence, ensure it uses them **correctly, clearly, and truthfully**. Penalize hallucinated or misleading use of evidence.
- **Low scores are not failures — they are essential to honest evaluation.** Do not hesitate to give them when needed.
- Be fair: if the RAG-GNN output is medically sound but differs from a reference diagnosis, that is acceptable if justified with clear reasoning and evidence.
- Common pitfalls to avoid:
   # Do not reward fluency over factual correctness
   # Penalize vague, unsupported, or hallucinated claims
   # Ensure retrieved evidence is **relevant**, **correctly interpreted**, and **not cherry-picked**
   # Penalize overconfidence when supporting evidence is weak or lacking

- For each category, use the format:

Category: [Name]
Score: X/Max
Explanation: [...]

- Ensure the **sum of the maximum possible points equals exactly 100**.

- Finish with the total score:

Final Score: XX/100
"""
    return prompt


In [ ]:
1. Input Preprocessing
Make sure the input is clean, structured, and ready:

Symptom Description: Normalize language (e.g., remove typos, standardize medical terms if needed)

Retrieved Contexts: Filter irrelevant or low-quality documents, chunk or trim if too long

GNN Output: Translate graph knowledge into readable structured insights (e.g., “fever → often linked with X based on GNN edges”)



In [ ]:
# import openai  # Or your model's API

# response = openai.ChatCompletion.create(
#     model="gpt-4",  # or a fine-tuned model like `gpt-3.5-medical`, or a local model
#     messages=[{"role": "user", "content": prompt}],
#     temperature=0.4,  # Low for factuality
#     max_tokens=1024
# )
# generated_diagnosis = response['choices'][0]['message']['content']


In [ ]:
def extract_final_diagnosis_score(text):
    match = re.search(r"Final Score:\s*(\d{1,3})\s*/\s*100", text)
    if match:
        return int(match.group(1))
    else:
        return None

def get_diagnosis_eval_data(df, model, gen_prompt_fn, eval_prompt_fn):
    diagnoses = []
    eval_scores = []

    for index, row in df.iterrows():
        generation_prompt = gen_prompt_fn(row["symptom_description"], row["retrieved_contexts"], row["gnn_knowledge"])
        diagnosis_output = None

        while diagnosis_output is None:
            try:
                diagnosis_response = model.generate_content(generation_prompt, generation_config=genai.types.GenerationConfig(temperature=1.2))
                diagnosis_output = diagnosis_response.text
            except Exception:
                time.sleep(2)

        diagnoses.append(diagnosis_output)

        evaluation_prompt = eval_prompt_fn(diagnosis_output, row["reference_diagnoses"], row["symptom_description"])
        score_output = None

        while score_output is None:
            try:
                score_response = model.generate_content(evaluation_prompt, generation_config=genai.types.GenerationConfig(temperature=1.2))
                score_output = score_response.text
            except Exception:
                time.sleep(2)

        score = extract_final_diagnosis_score(score_output)
        eval_scores.append(score)
        print(f"Case {index}: Final Score: {score}")

    return diagnoses, eval_scores

def run_diagnosis_evaluation(path, api_key, gen_prompt_fn, eval_prompt_fn):
    filename = os.path.basename(path)
    name_without_extension, _ = os.path.splitext(filename)

    df = pd.read_csv(path)

    os.environ["API_KEY"] = api_key
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-1.5-flash-latest")

    diagnoses, scores = get_diagnosis_eval_data(df, model, gen_prompt_fn, eval_prompt_fn)

    result_df = pd.DataFrame({
        "Symptom_Description": df["symptom_description"],
        "Reference_Diagnoses": df["reference_diagnoses"],
        "Generated_Diagnosis": diagnoses,
        "Diagnosis_Score": scores
    })

    csv_filename = f"{name_without_extension}_diagnosis_scores.csv"
    result_df.to_csv(csv_filename, index=False)

    print(f"\nResults saved to {csv_filename}")
    print(f"Average Score: {sum(scores)/len(scores):.2f}")

    return result_df, scores


In [ ]:
api = 'put your api key here'

In [ ]:
result_df, scores = run_diagnosis_evaluation(
    path="clinical_cases.csv",
    api_key=api_key,
    gen_prompt_fn=rag_gnn_generation_prompt,
    eval_prompt_fn=rag_gnn_evaluation_prompt
)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(scores, bins=10, edgecolor='black')
plt.title('Distribution of Evaluation Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

print(f"Mean Score: {sum(scores)/len(scores):.2f}")
print(f"Minimum: {min(scores)}, Maximum: {max(scores)}")
